In [ ]:
import json
import cPickle
import urllib2
import time
import csv
import twitter

# Key and Secret hidden for security.
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [ ]:
# Retrieve county names
with open('NYCountyNames.csv', 'r') as f:
    county_names = list(csv.reader(f))[0]

# Retrieve county locations and parse
with open('nycountiesstrippedmiles.csv', 'r') as f:
    lat_long_rad = list(csv.reader(f))
county_locs = [lat + ',' + lon + ',' + rad + 'mi'
               for (lat, lon, rad)
               in zip(lat_long_rad[0], lat_long_rad[1], lat_long_rad[2])]

# Geocodes (lat,lon,radius) indexed by the county name, but maybe
# add empty string index/location to simply get all such tweets
counties = dict(zip(county_names, county_locs))
# counties[''] = ''

# Hashtags to search
q = ['feelthebern',
     'sanders',
     'berniesanders',
     'bernie2016',
     'clinton',
     'hillaryclinton',
     'imwithher',
     'hillary2016',
     'tedcruz',
     'choosecruz',
     'cruzcrew',
     'unitewithcruz',
     'cruz2016',
     'trump2016',
     'donaldtrump',
     '#trump',
     'donaldtrump2016',
     'kasichcan',
     'johnkasich',
     'kasich2016',
     'kasich4us',
    ]

In [ ]:
# Search results will be stored herein. Within each county, we keep track
# of the tweets we've found, indexed by hashtag and stored in the list.
# We additionally keep a set of ids so we know when we find a tweet
# that we've already seen.
all_results = dict((county, dict((hashtag, ([], set())) for hashtag in q)) for county in counties)

In [ ]:
# Actually retrieve results. Twitter sometimes rejects due to
# rate limits; these are TwitterHTTPErrors. The connection also
# sometimes gets dropped, which is the URLError.
# Results are stored in pickles labeled with a number and the hashtag
for i in xrange(10000):
    for county in counties.keys():
        for hashtag in q:
            while True:
                try:
                    while True:
                        try:
                            results = twitter_api.search.tweets(q = hashtag,
                                                                geocode = counties[county],
                                                                count = 100) if county else twitter_api.search.tweets(q = hashtag,
                                                                                                                      count = 100)
                            break
                        except twitter.api.TwitterHTTPError:
                            print "Oops, Twitter said 'slow down.' Trying again in 1 minute."
                            time.sleep(60)
                    break
                except urllib2.URLError:
                    print "Oops, Twitter said 'no.' Trying again..."
            for num in xrange(len(results['statuses'])):
                stat_id = results['statuses'][num]['id']
                if stat_id not in all_results[county][hashtag][1]:
                    all_results[county][hashtag][0].append(results['statuses'][num])
                    all_results[county][hashtag][1].add(stat_id)
            while True:
                try:
                    while True:
                        try:
                            while True:
                                try:
                                    next_results = results['search_metadata']['next_results']
                                except KeyError, e: # No more results when next_results doesn't exist
                                    print "next_results doesn't exist"
                                    print county + hashtag + ': ' + str(len(all_results[county][hashtag][0])) + '/' + str(len(all_results[county][hashtag][1]))
                                    if len(all_results[county][hashtag][0]) > 0:
                                        with open('Tweets/' + county + '/' + hashtag + str(i) + '.p', 'wb') as f:
                                            cPickle.dump(all_results[county][hashtag][0], f)
                                        all_results[county][hashtag] = ([], all_results[county][hashtag][1])
                                    break

                                kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
                                #print kwargs
                                results = twitter_api.search.tweets(q = hashtag,
                                                                    geocode = counties[county],
                                                                    count = 100,
                                                                    max_id = kwargs['max_id']) if county else twitter_api.search.tweets(q = hashtag,
                                                                                                                                        count = '100',
                                                                                                                                        max_id = kwargs['max_id'])

                                for num in xrange(len(results['statuses'])):
                                    stat_id = results['statuses'][num]['id']
                                    if stat_id not in all_results[county][hashtag][1]:
                                        #print stat_id
                                        all_results[county][hashtag][0].append(results['statuses'][num])
                                        all_results[county][hashtag][1].add(stat_id)
                                if len(results['statuses']) < 100:
                                    print 'end of pages'
                                    print county + hashtag + ': ' + str(len(all_results[county][hashtag][0])) + '/' + str(len(all_results[county][hashtag][1]))
                                    if len(all_results[county][hashtag][0]) > 0:
                                        with open('Tweets/' + county + '/' + hashtag + str(i) + '.p', 'wb') as f:
                                            cPickle.dump(all_results[county][hashtag][0], f)
                                        all_results[county][hashtag] = ([], all_results[county][hashtag][1])
                                    break
                            break
                        except urllib2.URLError:
                            print "Oops, Twitter said 'no.' Trying again..."
                    break
                except twitter.api.TwitterHTTPError:
                    print "Oops, Twitter said 'slow down.' Trying again in 1 minute."
                    time.sleep(60)

In [ ]:
import os

# Code to make the directories we need
for county in counties:
    os.mkdir('Tweets/' + county)

# Cleaning code to combine various pickled files
# IOError might result if no results were returned for a hashtag at a certain i
# EOFError might result if loop was stopped while writing to a pickle
for county in counties:
    for hashtag in q:
        tag_results = []
        for i in xrange(9):
            try:
                try:
                    with open('Tweets/' + county + '/' + hashtag + str(i) + '.p', 'rb') as f:
                        tag_results += cPickle.load(f)
                except IOError:
                    continue
            except EOFError:
                print county
                print i
                continue
                
        if tag_results:
            with open('Tweets/' + county + '/' + hashtag + 'FULL' + '.p', 'wb') as f:
                cPickle.dump(tag_results, f)
                
# Cleaning code to remove various pickled files
for county in county_names:
    for hashtag in q:
        for i in xrange(9):
            try:
                os.remove('Tweets/' + county + '/' + hashtag + "FULL" + '.p')
            except WindowsError:
                continue